In [1]:
import numpy as np
import pandas as pd
import datetime
import random
import missingno as msno
import statistics
import numpy as np
import matplotlib.pyplot as plt
import math

In [2]:
#формирование датафрейма из скважин
#нужные парметры:проницаемость,эфф толщина, вязкость, пластвое и забойное 
#давление,радиус скважины и контура питания, расстояние между узлами, дебит узла
my_wells = pd.read_excel('DIPLOM_first_step.xlsx',header=[0],index_col=[0])

my_wells_list=my_wells['Well'].tolist()


FileNotFoundError: [Errno 2] No such file or directory: 'DIPLOM_first_step.xlsx'

In [ ]:
effective_thickness = pd.read_excel('new_well_info.xlsx',header=[0],index_col=[0])
all_parametres = pd.read_excel('new_production_td.xlsx',header=[0],index_col=[0])


effective_thickness.reset_index(inplace=True)
effective_thickness=effective_thickness.drop(['Thickness, meters','Coordinate X','Coordinate Y','Depth, meters'],axis=1)
effective_thickness=effective_thickness[effective_thickness.Well.isin(my_wells_list)].reset_index(drop=True)
effective_thickness = effective_thickness.drop(effective_thickness[effective_thickness['Effective thickness, meters'] >= 30.0].index)

effective_thickness.reset_index(inplace=True)
effective_thickness=effective_thickness.drop(['index'],axis=1)
effective_thickness

In [ ]:
my_wells_final=effective_thickness['Well'].tolist()

all_parametres.reset_index(inplace=True)
all_parametres=all_parametres[all_parametres.Well.isin(my_wells_final)].reset_index(drop=True)

all_parametres = all_parametres.drop(all_parametres[all_parametres.Date != 12.2017].index)
all_parametres.reset_index(inplace=True)
all_parametres=all_parametres.drop(['Oil rate, t/day','index','Reservoir','Date','Completion','Liquid rate, м3/сут','Watercut, %','Gas rate, m3/day','Reservoir temperature, °С','Skin','Water viscosity in reservoir conditions, cP','Liquid viscosity in reservoir conditions, cP','Oil density, g/cm2','Water density, g/cm3'],axis=1)

all_parametres

In [ ]:
all_parametres=all_parametres.groupby(by=['Well']).mean()
all_parametres.reset_index(inplace=True)
all_parametres = all_parametres.merge(effective_thickness, on='Well')
process_1=all_parametres['Well'].tolist()
all_parametres


In [ ]:
permeability = pd.read_excel('new_well_logs.xlsx',header=[0],index_col=[0])
permeability.reset_index(inplace=True)
permeability=permeability[permeability.Well.isin(process_1)].reset_index(drop=True)
permeability = permeability.drop(permeability[permeability['Permeabillity, mD'] == 0.0].index)
permeability=permeability.groupby(by=['Well']).mean()
permeability.reset_index(inplace=True)
permeability=permeability.drop(['Absolute thickness, m','Absolute depth, m','Thickness, m','Porosity, %','Oil saturation, %'],axis=1)
permeability

In [ ]:
final_data=all_parametres.merge(permeability, on='Well')

final_data = final_data.drop(final_data[final_data['Permeabillity, mD'] >= 40.0].index)

final_data

In [ ]:
reservoir_pressure=final_data['Reservoir pressure, atm'].tolist()
summa=0
for i in reservoir_pressure:
    summa=summa+i
final_data['Reservoir pressure, atm'] = final_data.apply(
    lambda x: 
    summa/len(reservoir_pressure), axis=1)
final_data['Well radius, m'] = final_data.apply(
    lambda x: 
    0.108, axis=1)
final_data['Supply contour radius, m'] = final_data.apply(
    lambda x: 
    500, axis=1)
final_data['Formation thickness, m'] = final_data.apply(
    lambda x: 
    287.1, axis=1)
final_data['Run to the bottom of formation, m'] = [49.82,40.59,35.06,27.68,25.83,22.14,20.3,18.45,16.61,14.76,14.76,14.76,14.76,14.76,14.76,14.76]
final_data['Distance between spheres, m'] = final_data.apply(
    lambda x: 
    100, axis=1)
final_data.reset_index(inplace=True)
perm_list=final_data['Permeabillity, mD'].tolist()
summa1=0
for i in perm_list:
    summa1=summa1+i
final_data['Permeabillity, mD'] = final_data.apply(
    lambda x: 
    summa1/len(perm_list), axis=1)
visc_list=final_data['Oil viscosity in reservoir conditions, cP'].tolist()
summa2=0
for i in visc_list:
    summa2=summa2+i
final_data['Oil viscosity in reservoir conditions, cP'] = final_data.apply(
    lambda x: 
    summa2/len(visc_list), axis=1)
final_data
#287.07272727272726

In [ ]:
final_data.to_excel('list_wells_test.xlsx')

In [ ]:
#вычисленик коэффициентов alpha m(бесконечный временной ряд)
#ДЛЯ ПЕРВОГО УЗЛА
x=[]
m=[-1,0 ,1]
for i in m:
    x.append(final_data['Formation thickness, m'][0]/(final_data['Well radius, m'][0]**(2)+
                                                       (-2*final_data['Formation thickness, m'][0]*
                                                       i)**(2))**(0.5)+
               final_data['Formation thickness, m'][0]/(final_data['Well radius, m'][0]**(2)+
                                                       (-2*final_data['Formation thickness, m'][0]*
                                                       i-2*final_data['Run to the bottom of formation, m'][0])**(2))**(0.5)-
                                                 final_data['Formation thickness, m'][0]/(final_data['Supply contour radius, m'][0]**(2)+
                                                       (-2*final_data['Formation thickness, m'][0]*
                                                       i)**(2))**(0.5)-
                 final_data['Formation thickness, m'][0]/(final_data['Supply contour radius, m'][0]**(2)+
                                                       (-2*final_data['Formation thickness, m'][0]*
                                                       i-2*final_data['Run to the bottom of formation, m'][0])**(2))**(0.5))
x
#БЕРЕМ  -1, 0 и 1 члены 

In [ ]:
#вычисленик коэффициентов alpha k(бесконечный временной ряд)
#ДЛЯ ПЕРВОГО УЗЛА
x=[]
m=[-15,-14,-13,-12,-11,-10,-9,-8,-7,-6,-5,-4,-3,-2,-1,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15]
for i in m:
    x.append(final_data['Formation thickness, m'][0]/(final_data['Supply contour radius, m'][0]**(2)+
                                                       (-2*final_data['Formation thickness, m'][0]*
                                                       i)**(2))**(0.5)+
               final_data['Formation thickness, m'][0]/(final_data['Supply contour radius, m'][0]**(2)+
                                                       (-2*final_data['Formation thickness, m'][0]*
                                                       i-2*final_data['Run to the bottom of formation, m'][0])**(2))**(0.5)-
                                                 final_data['Formation thickness, m'][0]/(final_data['Distance between spheres, m'][0]**(2)+
                                                       (-2*final_data['Formation thickness, m'][0]*
                                                       i)**(2))**(0.5)-
                 final_data['Formation thickness, m'][0]/(final_data['Distance between spheres, m'][0]**(2)+
                                                       (-2*final_data['Formation thickness, m'][0]*
                                                       i-2*final_data['Run to the bottom of formation, m'][0])**(2))**(0.5))
x
#до 15 члена

In [ ]:
#вычисление alpha k для каждого узла
final_data['Alpha_i_k_0'] = final_data.apply(
    lambda x: 
        
            x['Formation thickness, m']/(x['Supply contour radius, m']**(2)+
                                                       (-2*x['Formation thickness, m']*
                                                       0)**(2))**(0.5)+
               x['Formation thickness, m']/(x['Supply contour radius, m']**(2)+
                                                       (-2*x['Formation thickness, m']*
                                                       0-2*x['Run to the bottom of formation, m'])**(2))**(0.5)-
                                                 x['Formation thickness, m']/(x['Distance between spheres, m']**(2)+
                                                       (-2*x['Formation thickness, m']*
                                                       0)**(2))**(0.5)-
                 x['Formation thickness, m']/(x['Distance between spheres, m']**(2)+
                                                       (-2*x['Formation thickness, m']*
                                                       0-2*x['Run to the bottom of formation, m'])**(2))**(0.5) 
    
    , axis=1)
final_data['Alpha_i_k_1'] = final_data.apply(
    lambda x: 
        
            x['Formation thickness, m']/(x['Supply contour radius, m']**(2)+
                                                       (-2*x['Formation thickness, m']*
                                                       1)**(2))**(0.5)+
               x['Formation thickness, m']/(x['Supply contour radius, m']**(2)+
                                                       (-2*x['Formation thickness, m']*
                                                       1-2*x['Run to the bottom of formation, m'])**(2))**(0.5)-
                                                 x['Formation thickness, m']/(x['Distance between spheres, m']**(2)+
                                                       (-2*x['Formation thickness, m']*
                                                       1)**(2))**(0.5)-
                 x['Formation thickness, m']/(x['Distance between spheres, m']**(2)+
                                                       (-2*x['Formation thickness, m']*
                                                       1-2*x['Run to the bottom of formation, m'])**(2))**(0.5) 
    
    , axis=1)
final_data['Alpha_i_k_2'] = final_data.apply(
    lambda x: 
        
            x['Formation thickness, m']/(x['Supply contour radius, m']**(2)+
                                                       (-2*x['Formation thickness, m']*
                                                       2)**(2))**(0.5)+
               x['Formation thickness, m']/(x['Supply contour radius, m']**(2)+
                                                       (-2*x['Formation thickness, m']*
                                                       2-2*x['Run to the bottom of formation, m'])**(2))**(0.5)-
                                                 x['Formation thickness, m']/(x['Distance between spheres, m']**(2)+
                                                       (-2*x['Formation thickness, m']*
                                                       2)**(2))**(0.5)-
                 x['Formation thickness, m']/(x['Distance between spheres, m']**(2)+
                                                       (-2*x['Formation thickness, m']*
                                                       2-2*x['Run to the bottom of formation, m'])**(2))**(0.5) 
    
    , axis=1)
final_data['Alpha_i_k_3'] = final_data.apply(
    lambda x: 
        
            x['Formation thickness, m']/(x['Supply contour radius, m']**(2)+
                                                       (-2*x['Formation thickness, m']*
                                                       3)**(2))**(0.5)+
               x['Formation thickness, m']/(x['Supply contour radius, m']**(2)+
                                                       (-2*x['Formation thickness, m']*
                                                       3-2*x['Run to the bottom of formation, m'])**(2))**(0.5)-
                                                 x['Formation thickness, m']/(x['Distance between spheres, m']**(2)+
                                                       (-2*x['Formation thickness, m']*
                                                       3)**(2))**(0.5)-
                 x['Formation thickness, m']/(x['Distance between spheres, m']**(2)+
                                                       (-2*x['Formation thickness, m']*
                                                       3-2*x['Run to the bottom of formation, m'])**(2))**(0.5) 
    
    , axis=1)
final_data['Alpha_i_k_4'] = final_data.apply(
    lambda x: 
        
            x['Formation thickness, m']/(x['Supply contour radius, m']**(2)+
                                                       (-2*x['Formation thickness, m']*
                                                       4)**(2))**(0.5)+
               x['Formation thickness, m']/(x['Supply contour radius, m']**(2)+
                                                       (-2*x['Formation thickness, m']*
                                                       4-2*x['Run to the bottom of formation, m'])**(2))**(0.5)-
                                                 x['Formation thickness, m']/(x['Distance between spheres, m']**(2)+
                                                       (-2*x['Formation thickness, m']*
                                                       4)**(2))**(0.5)-
                 x['Formation thickness, m']/(x['Distance between spheres, m']**(2)+
                                                       (-2*x['Formation thickness, m']*
                                                       4-2*x['Run to the bottom of formation, m'])**(2))**(0.5) 
    
    , axis=1)
final_data['Alpha_i_k_5'] = final_data.apply(
    lambda x: 
        
            x['Formation thickness, m']/(x['Supply contour radius, m']**(2)+
                                                       (-2*x['Formation thickness, m']*
                                                       5)**(2))**(0.5)+
               x['Formation thickness, m']/(x['Supply contour radius, m']**(2)+
                                                       (-2*x['Formation thickness, m']*
                                                       5-2*x['Run to the bottom of formation, m'])**(2))**(0.5)-
                                                 x['Formation thickness, m']/(x['Distance between spheres, m']**(2)+
                                                       (-2*x['Formation thickness, m']*
                                                       5)**(2))**(0.5)-
                 x['Formation thickness, m']/(x['Distance between spheres, m']**(2)+
                                                       (-2*x['Formation thickness, m']*
                                                       5-2*x['Run to the bottom of formation, m'])**(2))**(0.5) 
    
    , axis=1)
final_data['Alpha_i_k_6'] = final_data.apply(
    lambda x: 
        
            x['Formation thickness, m']/(x['Supply contour radius, m']**(2)+
                                                       (-2*x['Formation thickness, m']*
                                                       6)**(2))**(0.5)+
               x['Formation thickness, m']/(x['Supply contour radius, m']**(2)+
                                                       (-2*x['Formation thickness, m']*
                                                       6-2*x['Run to the bottom of formation, m'])**(2))**(0.5)-
                                                 x['Formation thickness, m']/(x['Distance between spheres, m']**(2)+
                                                       (-2*x['Formation thickness, m']*
                                                       6)**(2))**(0.5)-
                 x['Formation thickness, m']/(x['Distance between spheres, m']**(2)+
                                                       (-2*x['Formation thickness, m']*
                                                       6-2*x['Run to the bottom of formation, m'])**(2))**(0.5) 
    
    , axis=1)
final_data['Alpha_i_k_7'] = final_data.apply(
    lambda x: 
        
            x['Formation thickness, m']/(x['Supply contour radius, m']**(2)+
                                                       (-2*x['Formation thickness, m']*
                                                       7)**(2))**(0.5)+
               x['Formation thickness, m']/(x['Supply contour radius, m']**(2)+
                                                       (-2*x['Formation thickness, m']*
                                                       7-2*x['Run to the bottom of formation, m'])**(2))**(0.5)-
                                                 x['Formation thickness, m']/(x['Distance between spheres, m']**(2)+
                                                       (-2*x['Formation thickness, m']*
                                                       7)**(2))**(0.5)-
                 x['Formation thickness, m']/(x['Distance between spheres, m']**(2)+
                                                       (-2*x['Formation thickness, m']*
                                                       7-2*x['Run to the bottom of formation, m'])**(2))**(0.5) 
    
    , axis=1)
final_data['Alpha_i_k_8'] = final_data.apply(
    lambda x: 
        
            x['Formation thickness, m']/(x['Supply contour radius, m']**(2)+
                                                       (-2*x['Formation thickness, m']*
                                                       8)**(2))**(0.5)+
               x['Formation thickness, m']/(x['Supply contour radius, m']**(2)+
                                                       (-2*x['Formation thickness, m']*
                                                       8-2*x['Run to the bottom of formation, m'])**(2))**(0.5)-
                                                 x['Formation thickness, m']/(x['Distance between spheres, m']**(2)+
                                                       (-2*x['Formation thickness, m']*
                                                       8)**(2))**(0.5)-
                 x['Formation thickness, m']/(x['Distance between spheres, m']**(2)+
                                                       (-2*x['Formation thickness, m']*
                                                       8-2*x['Run to the bottom of formation, m'])**(2))**(0.5) 
    
    , axis=1)
final_data['Alpha_i_k_9'] = final_data.apply(
    lambda x: 
        
            x['Formation thickness, m']/(x['Supply contour radius, m']**(2)+
                                                       (-2*x['Formation thickness, m']*
                                                       9)**(2))**(0.5)+
               x['Formation thickness, m']/(x['Supply contour radius, m']**(2)+
                                                       (-2*x['Formation thickness, m']*
                                                       9-2*x['Run to the bottom of formation, m'])**(2))**(0.5)-
                                                 x['Formation thickness, m']/(x['Distance between spheres, m']**(2)+
                                                       (-2*x['Formation thickness, m']*
                                                       9)**(2))**(0.5)-
                 x['Formation thickness, m']/(x['Distance between spheres, m']**(2)+
                                                       (-2*x['Formation thickness, m']*
                                                       9-2*x['Run to the bottom of formation, m'])**(2))**(0.5) 
    
    , axis=1)
final_data['Alpha_i_k_10'] = final_data.apply(
    lambda x: 
        
            x['Formation thickness, m']/(x['Supply contour radius, m']**(2)+
                                                       (-2*x['Formation thickness, m']*
                                                       10)**(2))**(0.5)+
               x['Formation thickness, m']/(x['Supply contour radius, m']**(2)+
                                                       (-2*x['Formation thickness, m']*
                                                       10-2*x['Run to the bottom of formation, m'])**(2))**(0.5)-
                                                 x['Formation thickness, m']/(x['Distance between spheres, m']**(2)+
                                                       (-2*x['Formation thickness, m']*
                                                       10)**(2))**(0.5)-
                 x['Formation thickness, m']/(x['Distance between spheres, m']**(2)+
                                                       (-2*x['Formation thickness, m']*
                                                       10-2*x['Run to the bottom of formation, m'])**(2))**(0.5) 
    
    , axis=1)
final_data['Alpha_i_k_11'] = final_data.apply(
    lambda x: 
        
            x['Formation thickness, m']/(x['Supply contour radius, m']**(2)+
                                                       (-2*x['Formation thickness, m']*
                                                       11)**(2))**(0.5)+
               x['Formation thickness, m']/(x['Supply contour radius, m']**(2)+
                                                       (-2*x['Formation thickness, m']*
                                                       11-2*x['Run to the bottom of formation, m'])**(2))**(0.5)-
                                                 x['Formation thickness, m']/(x['Distance between spheres, m']**(2)+
                                                       (-2*x['Formation thickness, m']*
                                                       11)**(2))**(0.5)-
                 x['Formation thickness, m']/(x['Distance between spheres, m']**(2)+
                                                       (-2*x['Formation thickness, m']*
                                                       11-2*x['Run to the bottom of formation, m'])**(2))**(0.5) 
    
    , axis=1)
final_data['Alpha_i_k_12'] = final_data.apply(
    lambda x: 
        
            x['Formation thickness, m']/(x['Supply contour radius, m']**(2)+
                                                       (-2*x['Formation thickness, m']*
                                                       12)**(2))**(0.5)+
               x['Formation thickness, m']/(x['Supply contour radius, m']**(2)+
                                                       (-2*x['Formation thickness, m']*
                                                       12-2*x['Run to the bottom of formation, m'])**(2))**(0.5)-
                                                 x['Formation thickness, m']/(x['Distance between spheres, m']**(2)+
                                                       (-2*x['Formation thickness, m']*
                                                       12)**(2))**(0.5)-
                 x['Formation thickness, m']/(x['Distance between spheres, m']**(2)+
                                                       (-2*x['Formation thickness, m']*
                                                       12-2*x['Run to the bottom of formation, m'])**(2))**(0.5) 
    
    , axis=1)
final_data['Alpha_i_k_13'] = final_data.apply(
    lambda x: 
        
            x['Formation thickness, m']/(x['Supply contour radius, m']**(2)+
                                                       (-2*x['Formation thickness, m']*
                                                       13)**(2))**(0.5)+
               x['Formation thickness, m']/(x['Supply contour radius, m']**(2)+
                                                       (-2*x['Formation thickness, m']*
                                                       13-2*x['Run to the bottom of formation, m'])**(2))**(0.5)-
                                                 x['Formation thickness, m']/(x['Distance between spheres, m']**(2)+
                                                       (-2*x['Formation thickness, m']*
                                                       13)**(2))**(0.5)-
                 x['Formation thickness, m']/(x['Distance between spheres, m']**(2)+
                                                       (-2*x['Formation thickness, m']*
                                                       13-2*x['Run to the bottom of formation, m'])**(2))**(0.5) 
    
    , axis=1)
final_data['Alpha_i_k_14'] = final_data.apply(
    lambda x: 
        
            x['Formation thickness, m']/(x['Supply contour radius, m']**(2)+
                                                       (-2*x['Formation thickness, m']*
                                                       14)**(2))**(0.5)+
               x['Formation thickness, m']/(x['Supply contour radius, m']**(2)+
                                                       (-2*x['Formation thickness, m']*
                                                       14-2*x['Run to the bottom of formation, m'])**(2))**(0.5)-
                                                 x['Formation thickness, m']/(x['Distance between spheres, m']**(2)+
                                                       (-2*x['Formation thickness, m']*
                                                       14)**(2))**(0.5)-
                 x['Formation thickness, m']/(x['Distance between spheres, m']**(2)+
                                                       (-2*x['Formation thickness, m']*
                                                       14-2*x['Run to the bottom of formation, m'])**(2))**(0.5) 
    
    , axis=1)
final_data['Alpha_i_k_15'] = final_data.apply(
    lambda x: 
        
            x['Formation thickness, m']/(x['Supply contour radius, m']**(2)+
                                                       (-2*x['Formation thickness, m']*
                                                       15)**(2))**(0.5)+
               x['Formation thickness, m']/(x['Supply contour radius, m']**(2)+
                                                       (-2*x['Formation thickness, m']*
                                                       15-2*x['Run to the bottom of formation, m'])**(2))**(0.5)-
                                                 x['Formation thickness, m']/(x['Distance between spheres, m']**(2)+
                                                       (-2*x['Formation thickness, m']*
                                                       15)**(2))**(0.5)-
                 x['Formation thickness, m']/(x['Distance between spheres, m']**(2)+
                                                       (-2*x['Formation thickness, m']*
                                                       15-2*x['Run to the bottom of formation, m'])**(2))**(0.5) 
    
    , axis=1)
final_data['Alpha_i_k_min_1'] = final_data.apply(
    lambda x: 
        
            x['Formation thickness, m']/(x['Supply contour radius, m']**(2)+
                                                       (-2*x['Formation thickness, m']*
                                                       (-1))**(2))**(0.5)+
               x['Formation thickness, m']/(x['Supply contour radius, m']**(2)+
                                                       (-2*x['Formation thickness, m']*
                                                       (-1)-2*x['Run to the bottom of formation, m'])**(2))**(0.5)-
                                                 x['Formation thickness, m']/(x['Distance between spheres, m']**(2)+
                                                       (-2*x['Formation thickness, m']*
                                                       (-1))**(2))**(0.5)-
                 x['Formation thickness, m']/(x['Distance between spheres, m']**(2)+
                                                       (-2*x['Formation thickness, m']*
                                                       (-1)-2*x['Run to the bottom of formation, m'])**(2))**(0.5) 
    
    , axis=1)
final_data['Alpha_i_k_min_2'] = final_data.apply(
    lambda x: 
        
            x['Formation thickness, m']/(x['Supply contour radius, m']**(2)+
                                                       (-2*x['Formation thickness, m']*
                                                       (-2))**(2))**(0.5)+
               x['Formation thickness, m']/(x['Supply contour radius, m']**(2)+
                                                       (-2*x['Formation thickness, m']*
                                                       (-2)-2*x['Run to the bottom of formation, m'])**(2))**(0.5)-
                                                 x['Formation thickness, m']/(x['Distance between spheres, m']**(2)+
                                                       (-2*x['Formation thickness, m']*
                                                       (-2))**(2))**(0.5)-
                 x['Formation thickness, m']/(x['Distance between spheres, m']**(2)+
                                                       (-2*x['Formation thickness, m']*
                                                       (-2)-2*x['Run to the bottom of formation, m'])**(2))**(0.5) 
    
    , axis=1)
final_data['Alpha_i_k_min_3'] = final_data.apply(
    lambda x: 
        
            x['Formation thickness, m']/(x['Supply contour radius, m']**(2)+
                                                       (-2*x['Formation thickness, m']*
                                                       (-3))**(2))**(0.5)+
               x['Formation thickness, m']/(x['Supply contour radius, m']**(2)+
                                                       (-2*x['Formation thickness, m']*
                                                       (-3)-2*x['Run to the bottom of formation, m'])**(2))**(0.5)-
                                                 x['Formation thickness, m']/(x['Distance between spheres, m']**(2)+
                                                       (-2*x['Formation thickness, m']*
                                                       (-3))**(2))**(0.5)-
                 x['Formation thickness, m']/(x['Distance between spheres, m']**(2)+
                                                       (-2*x['Formation thickness, m']*
                                                       (-3)-2*x['Run to the bottom of formation, m'])**(2))**(0.5) 
    
    , axis=1)
final_data['Alpha_i_k_min_4'] = final_data.apply(
    lambda x: 
        
            x['Formation thickness, m']/(x['Supply contour radius, m']**(2)+
                                                       (-2*x['Formation thickness, m']*
                                                       (-4))**(2))**(0.5)+
               x['Formation thickness, m']/(x['Supply contour radius, m']**(2)+
                                                       (-2*x['Formation thickness, m']*
                                                       (-4)-2*x['Run to the bottom of formation, m'])**(2))**(0.5)-
                                                 x['Formation thickness, m']/(x['Distance between spheres, m']**(2)+
                                                       (-2*x['Formation thickness, m']*
                                                       (-4))**(2))**(0.5)-
                 x['Formation thickness, m']/(x['Distance between spheres, m']**(2)+
                                                       (-2*x['Formation thickness, m']*
                                                       (-4)-2*x['Run to the bottom of formation, m'])**(2))**(0.5) 
    
    , axis=1)
final_data['Alpha_i_k_min_5'] = final_data.apply(
    lambda x: 
        
            x['Formation thickness, m']/(x['Supply contour radius, m']**(2)+
                                                       (-2*x['Formation thickness, m']*
                                                       (-5))**(2))**(0.5)+
               x['Formation thickness, m']/(x['Supply contour radius, m']**(2)+
                                                       (-2*x['Formation thickness, m']*
                                                       (-5)-2*x['Run to the bottom of formation, m'])**(2))**(0.5)-
                                                 x['Formation thickness, m']/(x['Distance between spheres, m']**(2)+
                                                       (-2*x['Formation thickness, m']*
                                                       (-5))**(2))**(0.5)-
                 x['Formation thickness, m']/(x['Distance between spheres, m']**(2)+
                                                       (-2*x['Formation thickness, m']*
                                                       (-5)-2*x['Run to the bottom of formation, m'])**(2))**(0.5) 
    
    , axis=1)
final_data['Alpha_i_k_min_6'] = final_data.apply(
    lambda x: 
        
            x['Formation thickness, m']/(x['Supply contour radius, m']**(2)+
                                                       (-2*x['Formation thickness, m']*
                                                       (-6))**(2))**(0.5)+
               x['Formation thickness, m']/(x['Supply contour radius, m']**(2)+
                                                       (-2*x['Formation thickness, m']*
                                                       (-6)-2*x['Run to the bottom of formation, m'])**(2))**(0.5)-
                                                 x['Formation thickness, m']/(x['Distance between spheres, m']**(2)+
                                                       (-2*x['Formation thickness, m']*
                                                       (-6))**(2))**(0.5)-
                 x['Formation thickness, m']/(x['Distance between spheres, m']**(2)+
                                                       (-2*x['Formation thickness, m']*
                                                       (-6)-2*x['Run to the bottom of formation, m'])**(2))**(0.5) 
    
    , axis=1)
final_data['Alpha_i_k_min_7'] = final_data.apply(
    lambda x: 
        
            x['Formation thickness, m']/(x['Supply contour radius, m']**(2)+
                                                       (-2*x['Formation thickness, m']*
                                                       (-7))**(2))**(0.5)+
               x['Formation thickness, m']/(x['Supply contour radius, m']**(2)+
                                                       (-2*x['Formation thickness, m']*
                                                       (-7)-2*x['Run to the bottom of formation, m'])**(2))**(0.5)-
                                                 x['Formation thickness, m']/(x['Distance between spheres, m']**(2)+
                                                       (-2*x['Formation thickness, m']*
                                                       (-7))**(2))**(0.5)-
                 x['Formation thickness, m']/(x['Distance between spheres, m']**(2)+
                                                       (-2*x['Formation thickness, m']*
                                                       (-7)-2*x['Run to the bottom of formation, m'])**(2))**(0.5) 
    
    , axis=1)
final_data['Alpha_i_k_min_8'] = final_data.apply(
    lambda x: 
        
            x['Formation thickness, m']/(x['Supply contour radius, m']**(2)+
                                                       (-2*x['Formation thickness, m']*
                                                       (-8))**(2))**(0.5)+
               x['Formation thickness, m']/(x['Supply contour radius, m']**(2)+
                                                       (-2*x['Formation thickness, m']*
                                                       (-8)-2*x['Run to the bottom of formation, m'])**(2))**(0.5)-
                                                 x['Formation thickness, m']/(x['Distance between spheres, m']**(2)+
                                                       (-2*x['Formation thickness, m']*
                                                       (-8))**(2))**(0.5)-
                 x['Formation thickness, m']/(x['Distance between spheres, m']**(2)+
                                                       (-2*x['Formation thickness, m']*
                                                       (-8)-2*x['Run to the bottom of formation, m'])**(2))**(0.5) 
    
    , axis=1)
final_data['Alpha_i_k_min_9'] = final_data.apply(
    lambda x: 
        
            x['Formation thickness, m']/(x['Supply contour radius, m']**(2)+
                                                       (-2*x['Formation thickness, m']*
                                                       (-9))**(2))**(0.5)+
               x['Formation thickness, m']/(x['Supply contour radius, m']**(2)+
                                                       (-2*x['Formation thickness, m']*
                                                       (-9)-2*x['Run to the bottom of formation, m'])**(2))**(0.5)-
                                                 x['Formation thickness, m']/(x['Distance between spheres, m']**(2)+
                                                       (-2*x['Formation thickness, m']*
                                                       (-9))**(2))**(0.5)-
                 x['Formation thickness, m']/(x['Distance between spheres, m']**(2)+
                                                       (-2*x['Formation thickness, m']*
                                                       (-9)-2*x['Run to the bottom of formation, m'])**(2))**(0.5) 
    
    , axis=1)
final_data['Alpha_i_k_min_10'] = final_data.apply(
    lambda x: 
        
            x['Formation thickness, m']/(x['Supply contour radius, m']**(2)+
                                                       (-2*x['Formation thickness, m']*
                                                       (-10))**(2))**(0.5)+
               x['Formation thickness, m']/(x['Supply contour radius, m']**(2)+
                                                       (-2*x['Formation thickness, m']*
                                                       (-10)-2*x['Run to the bottom of formation, m'])**(2))**(0.5)-
                                                 x['Formation thickness, m']/(x['Distance between spheres, m']**(2)+
                                                       (-2*x['Formation thickness, m']*
                                                       (-10))**(2))**(0.5)-
                 x['Formation thickness, m']/(x['Distance between spheres, m']**(2)+
                                                       (-2*x['Formation thickness, m']*
                                                       (-10)-2*x['Run to the bottom of formation, m'])**(2))**(0.5) 
    
    , axis=1)
final_data['Alpha_i_k_min_11'] = final_data.apply(
    lambda x: 
        
            x['Formation thickness, m']/(x['Supply contour radius, m']**(2)+
                                                       (-2*x['Formation thickness, m']*
                                                       (-11))**(2))**(0.5)+
               x['Formation thickness, m']/(x['Supply contour radius, m']**(2)+
                                                       (-2*x['Formation thickness, m']*
                                                       (-11)-2*x['Run to the bottom of formation, m'])**(2))**(0.5)-
                                                 x['Formation thickness, m']/(x['Distance between spheres, m']**(2)+
                                                       (-2*x['Formation thickness, m']*
                                                       (-11))**(2))**(0.5)-
                 x['Formation thickness, m']/(x['Distance between spheres, m']**(2)+
                                                       (-2*x['Formation thickness, m']*
                                                       (-11)-2*x['Run to the bottom of formation, m'])**(2))**(0.5) 
    
    , axis=1)
final_data['Alpha_i_k_min_12'] = final_data.apply(
    lambda x: 
        
            x['Formation thickness, m']/(x['Supply contour radius, m']**(2)+
                                                       (-2*x['Formation thickness, m']*
                                                       (-12))**(2))**(0.5)+
               x['Formation thickness, m']/(x['Supply contour radius, m']**(2)+
                                                       (-2*x['Formation thickness, m']*
                                                       (-12)-2*x['Run to the bottom of formation, m'])**(2))**(0.5)-
                                                 x['Formation thickness, m']/(x['Distance between spheres, m']**(2)+
                                                       (-2*x['Formation thickness, m']*
                                                       (-12))**(2))**(0.5)-
                 x['Formation thickness, m']/(x['Distance between spheres, m']**(2)+
                                                       (-2*x['Formation thickness, m']*
                                                       (-12)-2*x['Run to the bottom of formation, m'])**(2))**(0.5) 
    
    , axis=1)
final_data['Alpha_i_k_min_13'] = final_data.apply(
    lambda x: 
        
            x['Formation thickness, m']/(x['Supply contour radius, m']**(2)+
                                                       (-2*x['Formation thickness, m']*
                                                       (-13))**(2))**(0.5)+
               x['Formation thickness, m']/(x['Supply contour radius, m']**(2)+
                                                       (-2*x['Formation thickness, m']*
                                                       (-13)-2*x['Run to the bottom of formation, m'])**(2))**(0.5)-
                                                 x['Formation thickness, m']/(x['Distance between spheres, m']**(2)+
                                                       (-2*x['Formation thickness, m']*
                                                       (-13))**(2))**(0.5)-
                 x['Formation thickness, m']/(x['Distance between spheres, m']**(2)+
                                                       (-2*x['Formation thickness, m']*
                                                       (-13)-2*x['Run to the bottom of formation, m'])**(2))**(0.5) 
    
    , axis=1)
final_data['Alpha_i_k_min_14'] = final_data.apply(
    lambda x: 
        
            x['Formation thickness, m']/(x['Supply contour radius, m']**(2)+
                                                       (-2*x['Formation thickness, m']*
                                                       (-14))**(2))**(0.5)+
               x['Formation thickness, m']/(x['Supply contour radius, m']**(2)+
                                                       (-2*x['Formation thickness, m']*
                                                       (-14)-2*x['Run to the bottom of formation, m'])**(2))**(0.5)-
                                                 x['Formation thickness, m']/(x['Distance between spheres, m']**(2)+
                                                       (-2*x['Formation thickness, m']*
                                                       (-14))**(2))**(0.5)-
                 x['Formation thickness, m']/(x['Distance between spheres, m']**(2)+
                                                       (-2*x['Formation thickness, m']*
                                                       (-14)-2*x['Run to the bottom of formation, m'])**(2))**(0.5) 
    
    , axis=1)
final_data['Alpha_i_k_min_15'] = final_data.apply(
    lambda x: 
        
            x['Formation thickness, m']/(x['Supply contour radius, m']**(2)+
                                                       (-2*x['Formation thickness, m']*
                                                       (-15))**(2))**(0.5)+
               x['Formation thickness, m']/(x['Supply contour radius, m']**(2)+
                                                       (-2*x['Formation thickness, m']*
                                                       (-15)-2*x['Run to the bottom of formation, m'])**(2))**(0.5)-
                                                 x['Formation thickness, m']/(x['Distance between spheres, m']**(2)+
                                                       (-2*x['Formation thickness, m']*
                                                       (-15))**(2))**(0.5)-
                 x['Formation thickness, m']/(x['Distance between spheres, m']**(2)+
                                                       (-2*x['Formation thickness, m']*
                                                       (-15)-2*x['Run to the bottom of formation, m'])**(2))**(0.5) 
    
    , axis=1)
final_data['Alpha_i_k'] = final_data.apply(
    lambda x: 
    x['Alpha_i_k_min_15']+x['Alpha_i_k_min_14']+x['Alpha_i_k_min_13']+
    x['Alpha_i_k_min_12']+x['Alpha_i_k_min_11']+x['Alpha_i_k_min_10']+
    x['Alpha_i_k_min_9']+x['Alpha_i_k_min_8']+x['Alpha_i_k_min_7']+
    x['Alpha_i_k_min_6']+x['Alpha_i_k_min_5']+x['Alpha_i_k_min_4']+
    x['Alpha_i_k_min_3']+x['Alpha_i_k_min_2']+x['Alpha_i_k_min_1']+
    x['Alpha_i_k_0']+x['Alpha_i_k_1']+x['Alpha_i_k_2']+
    x['Alpha_i_k_3']+x['Alpha_i_k_4']+x['Alpha_i_k_5']+
    x['Alpha_i_k_6']+x['Alpha_i_k_8']+x['Alpha_i_k_10']+
    x['Alpha_i_k_7']+x['Alpha_i_k_9']+x['Alpha_i_k_11']+
    x['Alpha_i_k_12']+x['Alpha_i_k_14']+x['Alpha_i_k_15']+
    x['Alpha_i_k_13']
    , axis=1)
final_data = final_data.drop(['Alpha_i_k_min_15','Alpha_i_k_min_14','Alpha_i_k_min_13','Alpha_i_k_min_12','Alpha_i_k_min_11','Alpha_i_k_min_10','Alpha_i_k_min_9','Alpha_i_k_min_8','Alpha_i_k_min_7','Alpha_i_k_min_6','Alpha_i_k_min_5','Alpha_i_k_min_4','Alpha_i_k_min_3','Alpha_i_k_min_2','Alpha_i_k_min_1','Alpha_i_k_0','Alpha_i_k_1','Alpha_i_k_2','Alpha_i_k_3','Alpha_i_k_4','Alpha_i_k_5','Alpha_i_k_6','Alpha_i_k_7','Alpha_i_k_8','Alpha_i_k_9','Alpha_i_k_10','Alpha_i_k_11','Alpha_i_k_12','Alpha_i_k_13','Alpha_i_k_14','Alpha_i_k_15'],axis=1)
final_data

In [ ]:
#вычисление alpha m для каждого узла
final_data['Alpha_i_m_0'] = final_data.apply(
    lambda x: 
        
            x['Formation thickness, m']/(x['Well radius, m']**(2)+
                                                       (-2*x['Formation thickness, m']*
                                                       0)**(2))**(0.5)+
               x['Formation thickness, m']/(x['Well radius, m']**(2)+
                                                       (-2*x['Formation thickness, m']*
                                                       0-2*x['Run to the bottom of formation, m'])**(2))**(0.5)-
                                                 x['Formation thickness, m']/(x['Supply contour radius, m']**(2)+
                                                       (-2*x['Formation thickness, m']*
                                                       0)**(2))**(0.5)-
                 x['Formation thickness, m']/(x['Supply contour radius, m']**(2)+
                                                       (-2*x['Formation thickness, m']*
                                                       0-2*x['Run to the bottom of formation, m'])**(2))**(0.5) 
    
    , axis=1)
final_data['Alpha_i_m_1'] = final_data.apply(
    lambda x: 
        
            x['Formation thickness, m']/(x['Well radius, m']**(2)+
                                                       (-2*x['Formation thickness, m']*
                                                       1)**(2))**(0.5)+
               x['Formation thickness, m']/(x['Well radius, m']**(2)+
                                                       (-2*x['Formation thickness, m']*
                                                       1-2*x['Run to the bottom of formation, m'])**(2))**(0.5)-
                                                 x['Formation thickness, m']/(x['Supply contour radius, m']**(2)+
                                                       (-2*x['Formation thickness, m']*
                                                       1)**(2))**(0.5)-
                 x['Formation thickness, m']/(x['Supply contour radius, m']**(2)+
                                                       (-2*x['Formation thickness, m']*
                                                       1-2*x['Run to the bottom of formation, m'])**(2))**(0.5) 
    
    , axis=1)
final_data['Alpha_i_m_min_1'] = final_data.apply(
    lambda x: 
        
            x['Formation thickness, m']/(x['Well radius, m']**(2)+
                                                       (-2*x['Formation thickness, m']*
                                                       (-1))**(2))**(0.5)+
               x['Formation thickness, m']/(x['Well radius, m']**(2)+
                                                       (-2*x['Formation thickness, m']*
                                                       (-1)-2*x['Run to the bottom of formation, m'])**(2))**(0.5)-
                                                 x['Formation thickness, m']/(x['Supply contour radius, m']**(2)+
                                                       (-2*x['Formation thickness, m']*
                                                       (-1))**(2))**(0.5)-
                 x['Formation thickness, m']/(x['Supply contour radius, m']**(2)+
                                                       (-2*x['Formation thickness, m']*
                                                       (-1)-2*x['Run to the bottom of formation, m'])**(2))**(0.5) 
    
    , axis=1)
final_data['Alpha_i_m'] = final_data.apply(
    lambda x: 
    x['Alpha_i_m_min_1']+
    x['Alpha_i_m_0']+x['Alpha_i_m_1']
    , axis=1)
final_data = final_data.drop(['Alpha_i_m_0','Alpha_i_m_1','Alpha_i_m_min_1'],axis=1)
final_data

In [ ]:
#вычисление гидропроодности 
final_data['Flow capacity'] = final_data.apply(
    lambda x: 
    x['Permeabillity, mD']*
    x['Effective thickness, meters']/x['Oil viscosity in reservoir conditions, cP']
    , axis=1)
final_data

In [ ]:
#столбец свободных членов для СЛАУ + alpha i k/alpha i m
final_data['Constant term'] = final_data.apply(
    lambda x: 
    4*math.pi*x['Flow capacity']*
    x['Wellbore pressure, atm']/x['Alpha_i_m']-4*math.pi*x['Flow capacity']*
    x['Reservoir pressure, atm']/x['Alpha_i_m']
    , axis=1)
final_data['alpha/alpha'] = final_data.apply(
    lambda x: 
    x['Wellbore pressure, atm']/x['Alpha_i_m']
    , axis=1)
final_data

In [ ]:
#правая часть уравнения C и заготовка для матрицы левой части системы
B=final_data['alpha/alpha'].tolist()
C=final_data['Constant term'].tolist()
B_min=[]
for i in B:
    B_min.append(-i)
B_min

In [ ]:
#создание матрицы левой части уравнения
left_part_of_system=[]
for i in range(len(B_min)):
    copy = B_min[ : ]
    copy[i] += 1
    left_part_of_system.append(copy)
left_part_of_system

In [ ]:
#решение СЛАУ
A1=np.array(left_part_of_system)
B1=np.array(C)
X=np.linalg.solve(A1,B1)
Rate=X.tolist()
Oil_rate=[]
for i in Rate:
    Oil_rate.append(-i)
Oil_rate
final_data['Oil rate, t/day'] = [167.91351496109357, 241.86299809490293, 217.2441969465097, 133.42004742990946, 158.51940937434907, 110.70863907026458, 183.00195030970977, 214.3834091150673, 191.0237151250494, 112.48085445368017, 175.22210008367935, 198.69719879498402, 173.6991032546025, 162.34668917424725, 141.34836675412106, 220.9532893264759]
final_data = final_data.drop(['Alpha_i_k','Alpha_i_m','Flow capacity','Constant term','alpha/alpha'],axis=1)
final_data['Oil rate (surface), t/day'] = final_data.apply(
    lambda x: 
    x['Oil rate, t/day']/x['Fvf, m3/m3']
    , axis=1)
final_data

In [ ]:
Oil_rate

In [ ]:
prod=final_data['Oil rate (surface), t/day'].tolist()
sum=0
for i in prod:
    sum+=i
sum

In [ ]:
final_data.to_excel('production.xlsx')